# Exercise 5 <br>
Student: Duncan Ferguson <br>
Student Id: 871641260 <br>
Class: Comp 4431-1 <br>
Assignment: Exercise 5 <br>
Date: 10/16/2021 <br>
Group: Name: Broken Toe <br>
Group Members: Emma Bright, Mike Santoro <br>

In [21]:
import pandas as pd
from csv import reader
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

theData = [
['milk','onion','egg','bread','chili'],
['milk','egg','bread','chili'],
['milk','garlic','beans','chili'],
['milk','egg','beans','chili'],
['milk','onion','egg','bread'],
['milk','onion','egg'],
['garlic','onion','beans'],
['garlic','onion','beans'],
['garlic','onion','beans'],
['garlic','onion','beans'],
['chili','garlic','onion','beans'],
['chili','garlic','onion','beans'],
['garlic','onion','milk'],
['garlic','onion'],
['garlic','chili'],
['garlic','chili'],
['milk','onion','egg'],
['milk','onion','egg'],
['milk','onion','egg','bread'],
['milk','onion','chili','bread'],
]

# Use Transaction encoder to transform into an 1-hot boolean encoded numpy array
te = TransactionEncoder()
te_ary = te.fit(theData).transform(theData)

print("te_aray returned from Transactionencoder.fit().transform()")
print(te_ary)

te_aray returned from Transactionencoder.fit().transform()
[[False  True  True  True False  True  True]
 [False  True  True  True False  True False]
 [ True False  True False  True  True False]
 [ True False  True  True False  True False]
 [False  True False  True False  True  True]
 [False False False  True False  True  True]
 [ True False False False  True False  True]
 [ True False False False  True False  True]
 [ True False False False  True False  True]
 [ True False False False  True False  True]
 [ True False  True False  True False  True]
 [ True False  True False  True False  True]
 [False False False False  True  True  True]
 [False False False False  True False  True]
 [False False  True False  True False False]
 [False False  True False  True False False]
 [False False False  True False  True  True]
 [False False False  True False  True  True]
 [False  True False  True False  True  True]
 [False  True  True False False  True  True]]


In [22]:
# Convert into a dataframe for convenience and to pas into the apriori
df2 = pd.DataFrame(te_ary, columns=te.columns_)

print("DataFrame Version:")
print(df2.head(25))

DataFrame Version:
    beans  bread  chili    egg  garlic   milk  onion
0   False   True   True   True   False   True   True
1   False   True   True   True   False   True  False
2    True  False   True  False    True   True  False
3    True  False   True   True   False   True  False
4   False   True  False   True   False   True   True
5   False  False  False   True   False   True   True
6    True  False  False  False    True  False   True
7    True  False  False  False    True  False   True
8    True  False  False  False    True  False   True
9    True  False  False  False    True  False   True
10   True  False   True  False    True  False   True
11   True  False   True  False    True  False   True
12  False  False  False  False    True   True   True
13  False  False  False  False    True  False   True
14  False  False   True  False    True  False  False
15  False  False   True  False    True  False  False
16  False  False  False   True   False   True   True
17  False  False  False   T

In [23]:
# Call apriori to find frequent itemsets with min_support = 30%
freq_items = apriori(df2, min_support=0.3, use_colnames=True)
freq_items['length'] = freq_items['itemsets'].apply(lambda x: len(x))
freq_items.sort_values(['support', 'length'], inplace=True, ascending=False)
print("Frequent Items:")
print(freq_items)

Frequent Items:
    support                itemsets  length
5      0.75                 (onion)       1
3      0.55                (garlic)       1
4      0.55                  (milk)       1
1      0.45                 (chili)       1
8      0.40             (egg, milk)       2
10     0.40         (garlic, onion)       2
11     0.40           (milk, onion)       2
0      0.40                 (beans)       1
2      0.40                   (egg)       1
6      0.35         (beans, garlic)       2
12     0.30  (beans, garlic, onion)       3
13     0.30      (egg, milk, onion)       3
7      0.30          (beans, onion)       2
9      0.30            (egg, onion)       2


In [24]:
# Examples of How to filter your itemsets further, for illustration only, not actually used below
reducedFreqItems = freq_items[(((freq_items['length'] == 2) & (freq_items['support'] >= 0.40)) |
                               ((freq_items['length'] >= 3) & (freq_items['support'] >= 0.30 )))]

print("\n\nReduced freq_items (length == 2 & support >= 40%) | (length >=3 & support >= 30%) ")
print(reducedFreqItems)



Reduced freq_items (length == 2 & support >= 40%) | (length >=3 & support >= 30%) 
    support                itemsets  length
8       0.4             (egg, milk)       2
10      0.4         (garlic, onion)       2
11      0.4           (milk, onion)       2
12      0.3  (beans, garlic, onion)       3
13      0.3      (egg, milk, onion)       3


In [25]:
# Mining the rules by calling association_rules
print("The Rules")
rules = association_rules(freq_items, metric="confidence", min_threshold=0.7)
rules.columns = ['ante', 'consequ', 'antsup', 'consup', 'sup', 'conf', 'lift', 'leverage', 'convic']
print(rules[["ante","consequ","antsup","consup","sup","conf","lift"]])

The Rules
               ante          consequ  antsup  consup   sup      conf      lift
0             (egg)           (milk)    0.40    0.55  0.40  1.000000  1.818182
1            (milk)            (egg)    0.55    0.40  0.40  0.727273  1.818182
2          (garlic)          (onion)    0.55    0.75  0.40  0.727273  0.969697
3            (milk)          (onion)    0.55    0.75  0.40  0.727273  0.969697
4           (beans)         (garlic)    0.40    0.55  0.35  0.875000  1.590909
5   (beans, garlic)          (onion)    0.35    0.75  0.30  0.857143  1.142857
6    (beans, onion)         (garlic)    0.30    0.55  0.30  1.000000  1.818182
7   (garlic, onion)          (beans)    0.40    0.40  0.30  0.750000  1.875000
8           (beans)  (garlic, onion)    0.40    0.40  0.30  0.750000  1.875000
9       (egg, milk)          (onion)    0.40    0.75  0.30  0.750000  1.000000
10     (egg, onion)           (milk)    0.30    0.55  0.30  1.000000  1.818182
11    (milk, onion)            (egg)    0.

Questions 1: <br>
Using the values already set in the code  <br>
(i.e. apriori min_support = 0.3, association_rule min_threshold = 0.7) <br>
look at the result an answer: <br>

a). Which 1-itemset has the greatest support?

In [26]:
freq_items.sort_values(['support', 'length'], inplace=True, ascending=False)
print(freq_items[((freq_items['length'] == 1) & (freq_items['support'] == freq_items.support.max()))])

   support itemsets  length
5     0.75  (onion)       1


b). which 3-itemsets have suport >= 0.3

In [27]:
print(freq_items[((freq_items['length'] == 3) & (freq_items['support'] >= 0.3))])

    support                itemsets  length
12      0.3  (beans, garlic, onion)       3
13      0.3      (egg, milk, onion)       3


c). Why do the two rules (milk->egg and egg->milk) have different confidence values? <br>
The reason that (milk->egg and egg->milk) have different confidence values is because they have different
levels of support. There conditional probabilities are different.

In [28]:
print(freq_items[freq_items["itemsets"] == frozenset({'milk'})])
print(freq_items[freq_items["itemsets"] == frozenset({'egg'})])

   support itemsets  length
4     0.55   (milk)       1
   support itemsets  length
2      0.4    (egg)       1


d) Which rules are not "interesting"? Why do you say that?

The following all have a lift that is or below 1.

In [29]:
not_interesting = rules[["ante","consequ","antsup","consup","sup","conf","lift"]]
not_interesting[not_interesting["lift"] <= 1]

,ante,consequ,antsup,consup,sup,conf,lift
2,(garlic),(onion),0.55,0.75,0.4,0.727273,0.969697
3,(milk),(onion),0.55,0.75,0.4,0.727273,0.969697
9,"(egg, milk)",(onion),0.40,0.75,0.3,0.750000,1.000000
13,(beans),(onion),0.40,0.75,0.3,0.750000,1.000000
14,(egg),(onion),0.40,0.75,0.3,0.750000,1.000000


e) Two Rules have a lift value < 1.0 (garlic-> onion and milk->onion). Why are these especially useless rules?

Because there lift is below 1. Meaning that they do not have a coorelation

In [31]:
not_interesting[not_interesting["lift"] < .9999]

,ante,consequ,antsup,consup,sup,conf,lift
2,(garlic),(onion),0.55,0.75,0.4,0.727273,0.969697
3,(milk),(onion),0.55,0.75,0.4,0.727273,0.969697


2) The following file contains a clean version of ingredients in 256 indian dishes <br>
Modify the example code to read in and process this file.

Once read in, explore the file for various min_support and min_confidence settings.

Note it is suggested to start exploring with min_support = 1% or 2% and min_confidence = 30%
You can then parameter up/down and see what happens.